In [1]:

import findspark

findspark.init(spark_home='/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2',
                python_path='/Users/zhanghousu/conda/bin/python')

from pyspark import SparkContext
sc = SparkContext()
sc.addPyFile("jars/graphframes-0.8.2-spark2.4-s_2.11.jar")

                
from graphframes import *
from pyspark.sql.functions import *

23/04/29 16:39:57 WARN Utils: Your hostname, MacBook-Pro-279.local resolves to a loopback address: 127.0.0.1; using 10.79.140.225 instead (on interface en0)
23/04/29 16:39:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/29 16:39:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/29 16:39:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/29 16:39:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/29 16:39:59 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/29 16:39:59 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
links = sc.textFile('movie/links.csv')
ratings = sc.textFile('movie/ratings.csv')
movies_metadata = sc.textFile('movie/movies_metadata.csv')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
links_df = spark.read.csv(links, header=True, inferSchema=True, escape='"')
ratings_df = spark.read.csv(ratings, header=True, inferSchema=True, escape='"')
movies_metadata_df = spark.read.csv(movies_metadata, header=True, inferSchema=True, escape='"')

23/04/29 16:40:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 3): Attempting to kill Python Worker


23/04/29 16:40:26 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 26): Attempting to kill Python Worker


In [6]:
ra_id_rating = ratings_df.select('userId', 'movieId', 'rating')
ra_id_rating.show(5)

23/04/29 16:42:22 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 7 (TID 30): Attempting to kill Python Worker
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    110|   1.0|
|     1|    147|   4.5|
|     1|    858|   5.0|
|     1|   1221|   5.0|
|     1|   1246|   5.0|
+------+-------+------+
only showing top 5 rows



## This dataframe code will:

#### Create a GraphFrame using the ra_id_rating DataFrame.
#### Show the first 5 vertices and edges.
#### Calculate the in-degree and out-degree of each vertex.
#### Find the top 5 users who have rated the most movies.
#### Find the top 5 movies with the most ratings.

In [8]:
from graphframes import GraphFrame
from pyspark.sql.functions import col

# Prepare the vertices DataFrame
users_vertices = ra_id_rating.select("userId").distinct().withColumnRenamed("userId", "id")
movies_vertices = ra_id_rating.select("movieId").distinct().withColumnRenamed("movieId", "id")

vertices = users_vertices.union(movies_vertices)

# Prepare the edges DataFrame
edges = ra_id_rating.select("userId", "movieId", "rating").withColumnRenamed("userId", "src")\
                    .withColumnRenamed("movieId", "dst")

# Create a GraphFrame
graph = GraphFrame(vertices, edges)

# Show the vertices and edges
graph.vertices.show(5)
graph.edges.show(5)

# Calculate the in-degree of each vertex
in_degrees = graph.inDegrees
in_degrees.show(5)

# Calculate the out-degree of each vertex
out_degrees = graph.outDegrees
out_degrees.show(5)

# Find the top 5 users who have rated the most movies
top_raters = out_degrees.filter(col("id").isin(users_vertices.select("id").rdd.map(lambda row: row.id).collect()))\
                        .orderBy("outDegree", ascending=False)\
                        .limit(5)
top_raters.show()

# Find the top 5 movies with the most ratings
top_rated_movies = in_degrees.filter(col("id").isin(movies_vertices.select("id").rdd.map(lambda row: row.id).collect()))\
                             .orderBy("inDegree", ascending=False)\
                             .limit(5)
top_rated_movies.show()


/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+---+
| id|
+---+
|148|
|463|
|471|
|496|
|833|
+---+
only showing top 5 rows



23/04/29 16:45:08 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 76): Attempting to kill Python Worker
+---+----+------+
|src| dst|rating|
+---+----+------+
|  1| 110|   1.0|
|  1| 147|   4.5|
|  1| 858|   5.0|
|  1|1221|   5.0|
|  1|1246|   5.0|
+---+----+------+
only showing top 5 rows



/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-----+--------+
|   id|inDegree|
+-----+--------+
| 1645|   14346|
| 1591|    6317|
| 3175|   16216|
| 1580|   42193|
|68135|    2478|
+-----+--------+
only showing top 5 rows



...

23/04/29 16:48:33 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/04/29 16:48:47 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


23/04/29 16:48:48 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


+------+---------+
|    id|outDegree|
+------+---------+
| 45811|    18276|
|  8659|     9279|
|270123|     7638|
|179792|     7515|
|228291|     7410|
+------+---------+



+----+--------+
|  id|inDegree|
+----+--------+
| 356|   91921|
| 318|   91082|
| 296|   87901|
| 593|   84078|
|2571|   77960|
+----+--------+



In [11]:
from graphframes import GraphFrame

# Prepare the vertices DataFrame
imdb_vertices = links_df.select("imdbId").distinct().withColumnRenamed("imdbId", "id")
tmdb_vertices = links_df.select("tmdbId").distinct().withColumnRenamed("tmdbId", "id")

vertices = imdb_vertices.union(tmdb_vertices)

# Prepare the edges DataFrame
edges = links_df.select("imdbId", "tmdbId").withColumnRenamed("imdbId", "src").withColumnRenamed("tmdbId", "dst")

# Create a GraphFrame
graph = GraphFrame(vertices, edges)

# Show the vertices and edges
graph.vertices.show(5)
graph.edges.show(5)

# Calculate the degree of each vertex
degrees = graph.degrees
degrees.show(5)


/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+------+
|    id|
+------+
|110081|
|109068|
|117500|
| 94265|
|106544|
+------+
only showing top 5 rows

+------+-----+
|   src|  dst|
+------+-----+
|114709|  862|
|113497| 8844|
|113228|15602|
|114885|31357|
|113041|11862|
+------+-----+
only showing top 5 rows



/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+------+------+
|    id|degree|
+------+------+
|110081|     1|
|109068|     1|
| 11858|     1|
| 36355|     1|
|117500|     2|
+------+------+
only showing top 5 rows



This result shows some basic information about the GraphFrame created using the links_df dataset. We can see that the vertices DataFrame is only showing the top 5 vertices, which includes the imdbId data. The edges DataFrame displays the top 5 mappings between imdbId and tmdbId, i.e., the source vertex (src) and the target vertex (dst).

Finally, we calculated the degree of each vertex, and here we are showing the top 5 vertices' degrees. The degree represents the number of edges related to the vertex. In this example, we can see that the degree varies between 1 and 2. This indicates that in our dataset, the association between most imdbId and tmdbId is relatively weak, with only a few imdbId being associated with multiple tmdbId.